In [1]:
import cv2
import argparse
import os
from pathlib import Path
from PIL import Image
from mtcnn import MTCNN
from datetime import datetime

import torch
from torchvision import transforms as trans

from PIL import Image
import numpy as np
from mtcnn_pytorch.src.align_trans import get_reference_facial_points, warp_and_crop_face

In [2]:
from model.model import MobileFaceNet

from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
import sklearn
from scipy import interpolate
import datetime

import matplotlib.pyplot as plt

In [3]:
device = torch.device('cuda')
embedding_size = 512
model = MobileFaceNet(embedding_size).to(device)
model.load_state_dict(torch.load('./model_mobilefacenet.pth'))
model.eval()

MobileFaceNet(
  (conv1): Conv_block(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=64)
  )
  (conv2_dw): Conv_block(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=64)
  )
  (conv_23): Depth_Wise(
    (conv): Conv_block(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=128)
    )
    (conv_dw): Conv_block(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128, bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [5]:
mtcnn = MTCNN()
print('mtcnn loaded')

frame_count = 0
face_limit = 10

mtcnn loaded


In [8]:
def prepare_facebank(model, mtcnn, img_dir, txtfile, device, is_same=True):
    model.eval()
    embeddings =  []
    names = []
    embs = []
    
    test_transform = trans.Compose([
                    trans.ToTensor(),
                    trans.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                ])
    
    f = open(txtfile, "r")
    for line in f.readlines():
        #print(line)
        cls, filename = line.split()
#         print(cls)
#         print(filename)
        
        img = os.path.join(img_dir, filename)
        print(img)
        img = Image.open(img)
        img = mtcnn.align(img)
        with torch.no_grad():
            embeddings.append(model(test_transform(img).to(device).unsqueeze(0)).cpu())
            names.append(cls)
        #embedding = torch.cat(embs).mean(0,keepdim=True)
        #embeddings.append(embedding)        
    f.close()
    
    embeddings = torch.cat(embeddings)
    names = np.array(names)
    if is_same: key = 'positive'
    else: key = 'negative'
    torch.save(embeddings, './{}.pth'.format(key))
    np.save('./{}_names'.format(key), names)
    return embeddings, names

In [9]:
# img_dir = './Dataset/celebA/img_align_celeba'
# txtfile = './shen_celeba_positive_mod.txt'
# is_same = True
# prepare_facebank(model, mtcnn, img_dir, txtfile, device, is_same)

D:/002.Projects/01.Dataset/celebA/img_align_celeba\024091.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\038345.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\050215.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\064595.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\010376.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\044855.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\004891.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\149002.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\031683.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\049608.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\064382.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\060881.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\008330.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\024069.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\003968.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\026636.jpg
D:/002.P

(tensor([[ 0.0131, -0.0406,  0.0649,  ...,  0.0461,  0.0255, -0.0145],
         [-0.0077,  0.0398,  0.0046,  ...,  0.0090,  0.0321, -0.0601],
         [-0.0232, -0.0378, -0.0317,  ...,  0.0669,  0.0914,  0.0580],
         ...,
         [-0.0008,  0.0221, -0.0453,  ..., -0.0024, -0.0107, -0.0681],
         [ 0.0074, -0.0194, -0.0065,  ...,  0.0139, -0.0088, -0.0028],
         [-0.0235, -0.0238,  0.0417,  ..., -0.0150,  0.0514,  0.0529]]),
 array(['1', '10', '101', '102', '103', '105', '107', '11', '110', '112',
        '113', '115', '116', '117', '118', '119', '12', '121', '122', '13',
        '14', '15', '17', '19', '20', '21', '22', '23', '24', '25', '27',
        '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '38',
        '39', '4', '40', '41', '42', '43', '45', '46', '47', '48', '49',
        '5', '50', '51', '52', '53', '54', '55', '57', '58', '59', '6',
        '60', '61', '63', '64', '65', '66', '67', '68', '69', '7', '70',
        '71', '73', '77', '78', '79', '8', 

In [10]:
# img_dir = './Dataset/celebA/img_align_celeba'
# txtfile = './shen_celeba_negative_mod.txt'
# is_same = False
# prepare_facebank(model, mtcnn, img_dir, txtfile, device, is_same)

D:/002.Projects/01.Dataset/celebA/img_align_celeba\062504.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\053823.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\011141.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\070410.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\038682.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\035079.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\012357.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\003668.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\000854.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\006588.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\064367.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\131952.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\050831.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\027514.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\008838.jpg
D:/002.Projects/01.Dataset/celebA/img_align_celeba\052773.jpg
D:/002.P

(tensor([[ 0.0828, -0.0412, -0.0044,  ...,  0.0045,  0.0433, -0.0261],
         [ 0.0227,  0.0418, -0.0084,  ...,  0.0012, -0.0072,  0.0255],
         [-0.0104, -0.0419, -0.0025,  ..., -0.0320,  0.0585, -0.0299],
         ...,
         [-0.0275, -0.0259, -0.0644,  ..., -0.0461, -0.0334, -0.0836],
         [-0.0288, -0.0225, -0.0113,  ...,  0.0090,  0.0100, -0.0882],
         [-0.0467, -0.0074, -0.0064,  ..., -0.0383,  0.0497,  0.0179]]),
 array(['1', '10', '101', '102', '103', '105', '107', '11', '110', '112',
        '113', '115', '116', '117', '118', '119', '12', '121', '122', '13',
        '14', '15', '17', '19', '20', '21', '22', '23', '24', '25', '27',
        '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '38',
        '39', '4', '40', '41', '42', '43', '45', '46', '47', '48', '49',
        '5', '50', '51', '52', '53', '54', '55', '57', '58', '59', '6',
        '60', '61', '63', '64', '65', '66', '67', '68', '69', '7', '70',
        '71', '73', '77', '78', '79', '8', 

In [26]:
def calculate_roc(thresholds, embeddings1, embeddings2, actual_issame, nrof_folds=10, pca=0):
    assert (embeddings1.shape[0] == embeddings2.shape[0])
    assert (embeddings1.shape[1] == embeddings2.shape[1])
    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    nrof_thresholds = len(thresholds)
    k_fold = KFold(n_splits=nrof_folds, shuffle=False)

    tprs = np.zeros((nrof_folds, nrof_thresholds))
    fprs = np.zeros((nrof_folds, nrof_thresholds))
    accuracy = np.zeros((nrof_folds))
    best_thresholds = np.zeros((nrof_folds))
    indices = np.arange(nrof_pairs)
    # print('pca', pca)

    if pca == 0:
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff), 1)

    for fold_idx, (train_set, test_set) in enumerate(k_fold.split(indices)):
        # print('train_set', train_set)
        # print('test_set', test_set)
        if pca > 0:
            print('doing pca on', fold_idx)
            embed1_train = embeddings1[train_set]
            embed2_train = embeddings2[train_set]
            _embed_train = np.concatenate((embed1_train, embed2_train), axis=0)
            # print(_embed_train.shape)
            pca_model = PCA(n_components=pca)
            pca_model.fit(_embed_train)
            embed1 = pca_model.transform(embeddings1)
            embed2 = pca_model.transform(embeddings2)
            embed1 = sklearn.preprocessing.normalize(embed1)
            embed2 = sklearn.preprocessing.normalize(embed2)
            # print(embed1.shape, embed2.shape)
            diff = np.subtract(embed1, embed2)
            dist = np.sum(np.square(diff), 1)

        # Find the best threshold for the fold
        acc_train = np.zeros((nrof_thresholds))
        for threshold_idx, threshold in enumerate(thresholds):
            _, _, acc_train[threshold_idx] = calculate_accuracy(threshold, dist[train_set], actual_issame[train_set])
        best_threshold_index = np.argmax(acc_train)
#         print('best_threshold_index', best_threshold_index, acc_train[best_threshold_index])
        best_thresholds[fold_idx] = thresholds[best_threshold_index]
        for threshold_idx, threshold in enumerate(thresholds):
            tprs[fold_idx, threshold_idx], fprs[fold_idx, threshold_idx], _ = calculate_accuracy(threshold,
                                                                                                 dist[test_set],
                                                                                                 actual_issame[
                                                                                                     test_set])
        _, _, accuracy[fold_idx] = calculate_accuracy(thresholds[best_threshold_index], dist[test_set],
                                                      actual_issame[test_set])

    tpr = np.mean(tprs, 0)
    fpr = np.mean(fprs, 0)
    return tpr, fpr, accuracy, best_thresholds


def calculate_accuracy(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    fp = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))
    fn = np.sum(np.logical_and(np.logical_not(predict_issame), actual_issame))

    tpr = 0 if (tp + fn == 0) else float(tp) / float(tp + fn)
    fpr = 0 if (fp + tn == 0) else float(fp) / float(fp + tn)
    acc = float(tp + tn) / dist.size
    return tpr, fpr, acc

def calculate_val_far(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    # true_accept = np.sum(np.logical_and(predict_issame, actual_issame))
    false_accept = np.sum(np.logical_and(
        predict_issame, np.logical_not(actual_issame)))
    false_reject = np.sum(np.logical_and(
        np.logical_not(predict_issame), actual_issame))
    # true_reject = np.sum(np.logical_and(
    #     np.logical_not(actual_issame), np.logical_not(predict_issame)))
    n_same = np.sum(actual_issame)
    n_diff = np.sum(np.logical_not(actual_issame))
    # val = float(true_accept) / float(n_same)
    far = 100*float(false_accept) / float(n_diff)
    frr = 100*float(false_reject) / float(n_same)
    return far, frr

def plot_EER(far, frr, threshold, thres, apath, epoch):
    fig = plt.figure()
    plt.plot(threshold, far, color='g', label='FAR')
    plt.plot(threshold, frr, color='y', label='FRR')
    py = far[thres]
    px = threshold[thres]
    plt.plot(px, py, 
        'ro', label='TH: {:.4f},\nEER: {:.4f}'.format(px,py))
    # Adding labels and titles to the plot
    plt.xlabel('Threshold Distance')
    plt.ylabel('Rate (%)')
    plt.title('Performance at Epoch {}'.format(epoch))
    plt.grid(True)
    legend = plt.legend(loc='best', fontsize='x-large')
    legend.get_frame().set_facecolor('#ffffff')
    # plt.show()
    plt.savefig(os.path.join(apath, 'performance_ep{}.pdf'.format(epoch)))
    plt.close(fig)

In [14]:
p_targets = torch.load('./positive.pth')
p_names = np.load('./positive_names.npy')

n_targets = torch.load('./negative.pth')
n_names = np.load('./negative_names.npy')

In [ ]:
mspl_results = 'celeba_results'
save_name = 'celeba_results'

embedding1 = []
embedding2 = []
p_emb = []
n_emb = []

positive_embs= []
negative_embs = []
test_transform = trans.Compose([
                    trans.ToTensor(),
                    trans.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                ])
distnace = []   
for imgs in os.listdir(mspl_results):
    img = os.path.join(mspl_results, imgs)
    filename = os.path.splitext(os.path.split(img)[-1])[0]
    cls = filename.split('_')[0]
    cls = int(cls)
    tar_idx = np.where(p_names==str(cls))

    # print(img, tar_idx)
    img = Image.open(img)
    img = mtcnn.align(img)
    if img is not None:        
        with torch.no_grad():
            #model(test_transform(img).to(device).unsqueeze(0)).cpu()
            feat = model(test_transform(img).to(device).unsqueeze(0)).cpu()            
            # print(len(feat))
            embedding1.append(feat)   
            p_emb.append(p_targets[tar_idx].cpu())
            n_emb.append(n_targets[tar_idx].cpu())
            

embedding1 = embedding1 + embedding1
embedding2 = p_emb + n_emb
assert len(embedding1) == len(embedding1)

issame = []
for _ in range(len(p_emb)):
    issame.append(True)
for _ in range(len(n_emb)):
    issame.append(False)
assert len(embedding1) == len(embedding1) == len(issame)

C:\Users\LTB\Desktop\aaa\mtcnn_pytorch\src\matlab_cp2tform.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)


In [ ]:
feat_dist = []
for e1, e2 in zip(embedding1, embedding2):
    dist = (e1 - e2).norm(p=2, dim=1)
    dist = dist.numpy()
    feat_dist.append(dist[0])

In [ ]:
thresholds = np.arange(0, 4, 0.0001)

far = []
frr = []
total = []

for _, threshold in enumerate(thresholds):
    far_val, frr_val = calculate_val_far(threshold, feat_dist, issame)
    far.append(far_val)
    frr.append(frr_val)
    total.append(far_val+frr_val)
err_list = np.subtract(far, frr)
err_inter = interpolate.interp1d(err_list, thresholds, kind = 'slinear')
th = err_inter(0.0)
smallest = int(np.min(np.where(thresholds >= th)))

In [ ]:
fig = plt.figure()
plt.plot(thresholds, far, color='r', label='FAR')
plt.plot(thresholds, frr, color='b', label='FRR')
py = far[smallest]
px = thresholds[smallest]
plt.plot(px, py,  'ro', label='TH: {:.4f},\nEER: {:.4f}'.format(px,py))
# Adding labels and titles to the plot
plt.xlabel('Threshold Distance')
plt.ylabel('Rate (%)')
#plt.title('{} Performance_200401'.format(modelname))
plt.grid(True)
legend = plt.legend(loc='best', fontsize='x-large')
legend.get_frame().set_facecolor('#ffffff')
plt.savefig('{}.png'.format(save_name))
plt.show()
plt.close(fig)

In [ ]:
em1 = []
em2 = []
for e1, e2 in zip(embedding1, embedding2):
    em1.append(e1.numpy()[0])
    em2.append(e2.numpy()[0])

In [ ]:
len(em1[0])

In [ ]:
thresholds = np.arange(0, 4, 0.0001)
tpr, fpr, accuracy, best_thresholds = calculate_roc(thresholds, np.asarray(em1), np.asarray(em2), np.asarray(issame))

In [ ]:
print(accuracy.mean())

In [ ]:
"""Create a ROC pyplot plot and save"""
plt.figure()
plt.xlabel("FPR", fontsize=14)
plt.ylabel("TPR", fontsize=14)
plt.title("ROC Curve", fontsize=14)
plot = plt.plot(fpr, tpr, linewidth=2)
plt.savefig('{}_ROC.png'.format(save_name))
plt.show()
plt.close()